In [84]:
import fiona
from shapely.geometry import box
import folium
from shapely.geometry import Point
import os
import time
import pyogrio
import geopandas as gpd
import geopandas as gpd
import pyogrio
from shapely.geometry import Point
import folium
import pyogrio
import geopandas as gpd
import folium
from shapely.geometry import Point
from IPython.display import display
import geopandas as gpd
from IPython.display import display

## Antall rader

In [85]:


ar50_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb"

gdf = gpd.read_file(ar50_file)

print(gdf.geom_type.value_counts())  # Teller Polygon og MultiPolygon
print(len(gdf))  # Sjekker total antall rader


MultiPolygon    176214
Name: count, dtype: int64
176214


## Konvertering til FGB (GML og GDB)

In [86]:

def convert_ar50_to_fgb(input_file, output_file):
    """
    Konverterer AR50-data fra GML eller GDB til FlatGeobuf (FGB),
    og filtrerer kun ut polygon-geometrier.
    """
    file_ext = os.path.splitext(input_file)[1].lower()

    # Bruk riktig lag for hver filtype
    if file_ext == ".gml":
        layer_name = "ArealressursFlate"
        gdf = gpd.read_file(input_file, layer=layer_name)

    elif file_ext == ".gdb":
        layer_name = "ar50"
        if layer_name not in fiona.listlayers(input_file):
            raise ValueError(f"Laget '{layer_name}' finnes ikke i GDB-filen.")
        gdf = gpd.read_file(input_file, layer=layer_name)

    else:
        raise ValueError(f"Ukjent eller ikke støttet filtype: {file_ext}")

    # Filtrer kun polygoner
    gdf = gdf[gdf.geom_type.isin(['Polygon', 'MultiPolygon'])]

    # Lagre som FlatGeobuf
    gdf.to_file(output_file, driver="FlatGeobuf")
    print(f"Lagret '{layer_name}' som {output_file}")

    return gdf

## Velg hvilke filer du vil konvertere fra GML eller GBD til FGB

In [87]:

# Eksempelbruk
input_gml = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gml.gml"
input_gdb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gdb.gdb"

# Konverter GML til FGB
output_fgb_gml = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gml.fgb"
convert_ar50_to_fgb(input_gml, output_fgb_gml)

# Konverter GDB til FGB
output_fgb_gdb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb"
convert_ar50_to_fgb(input_gdb, output_fgb_gdb)


Lagret 'ArealressursFlate' som /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gml.fgb
Lagret 'ar50' som /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb


,rid,kilde,artype,arskogbon,artreslag,arjordbr,arveget,arkartstd,komid,fylkeid,geometry
0,1,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,52,AR50,4206,42,"MULTIPOLYGON (((357097.324 6444797.96, 357057...."
1,2,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,20,98,98,24,98,AR50,4206,42,"MULTIPOLYGON (((357232.182 6444717.404, 357231..."
2,3,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,55,AR50,4206,42,"MULTIPOLYGON (((357578.261 6444906.04, 357585...."
3,4,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,30,13,31,98,98,AR50,4206,42,"MULTIPOLYGON (((357998.132 6441329.147, 358005..."
4,5,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,60,11,39,98,98,AR50,4206,42,"MULTIPOLYGON (((358617.14 6442580.943, 358631...."
...,...,...,...,...,...,...,...,...,...,...,...
176209,176210,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,30,11,31,98,98,AR50,4212,42,"MULTIPOLYGON (((485458.32 6520535.79, 485484.5..."
176210,176211,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,60,11,39,98,98,AR50,4212,42,"MULTIPOLYGON (((483990.429 6519664.481, 483991..."
176211,176212,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,60,12,31,98,98,AR50,4212,42,"MULTIPOLYGON (((486986.352 6519317.402, 486986..."
176212,176213,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,30,11,99,98,98,AR50,4212,42,"MULTIPOLYGON (((485607.834 6519514.217, 485623..."


## Leser filen

In [88]:

# Sti til din FlatGeobuf (.fgb) fil
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb"  # Endre til din filsti

# Les inn FGB-filen
gdf = gpd.read_file(fgb_file)

# Vis de 10 første radene som en tabell
gdf.head(10)


,rid,kilde,artype,arskogbon,artreslag,arjordbr,arveget,arkartstd,komid,fylkeid,geometry
0,97254,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,99,98,98,98,98,AR50,4202,42,"MULTIPOLYGON (((499330.268 6451352.944, 499395..."
1,137183,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,99,98,98,98,98,AR50,4215,42,"MULTIPOLYGON (((463337.546 6419930.693, 463427..."
2,50982,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,99,98,98,98,98,AR50,4204,42,"MULTIPOLYGON (((455377.855 6416267.632, 455468..."
3,50981,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,82,98,98,98,98,AR50,4204,42,"MULTIPOLYGON (((459362.812 6418093.267, 459272..."
4,135365,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,99,AR50,4215,42,"MULTIPOLYGON (((455999.361 6442165.662, 456015..."
5,134168,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,51,AR50,4215,42,"MULTIPOLYGON (((455969.733 6442883.25, 455964...."
6,134174,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,52,AR50,4215,42,"MULTIPOLYGON (((455842.562 6442434.745, 455822..."
7,135360,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,54,AR50,4215,42,"MULTIPOLYGON (((455599.141 6442677.93, 455612...."
8,135361,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,51,AR50,4215,42,"MULTIPOLYGON (((455429.774 6442400.392, 455427..."
9,134175,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,52,AR50,4215,42,"MULTIPOLYGON (((454851.4 6442824.725, 454871.8..."


## Viser kolonnene i filen

In [90]:

input_fgb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb"

gdf = gpd.read_file(input_fgb)
print("Tilgjengelige kolonner:", gdf.columns.tolist())


Tilgjengelige kolonner: ['rid', 'kilde', 'artype', 'arskogbon', 'artreslag', 'arjordbr', 'arveget', 'arkartstd', 'komid', 'fylkeid', 'geometry']


## Bruker partial read for å lese filen, så lastes ned den nyfiltrerte.  "bruker partial read med pyogrio.read_dataframe() for å lese kun den kolonnen du skal filtrere på + geometrien."

In [91]:


def sjekk_filstørrelse(filsti):
    """Returnerer filstørrelse i MB eller KB."""
    if not os.path.exists(filsti):
        return "Fil ikke funnet"
    størrelse = os.path.getsize(filsti)
    return f"{størrelse / (1024 * 1024):.2f} MB" if størrelse > 100_000 else f"{størrelse / 1024:.2f} KB"

def les_fil_med_tid(filsti, kolonne=None):
    """Leser GeoDataFrame med tid, støtter partial read ved spesifikk kolonne."""
    start = time.time()

    if kolonne:
        gdf = pyogrio.read_dataframe(filsti, columns=[kolonne])  # Kun én kolonne + geometri
    else:
        gdf = gpd.read_file(filsti)  # Fallback til vanlig lesing

    slutt = time.time()
    lesetid = slutt - start
    return gdf, lesetid

def filtrer_og_sammenlign(input_fgb, kolonne, verdi, output_fgb):
    print(f"Leser originalfil (kun kolonne '{kolonne}') fra: {input_fgb}")
    gdf_org, tid_org = les_fil_med_tid(input_fgb, kolonne=kolonne)

    print(f"\nFiltrerer på: {kolonne} = {verdi}")
    gdf_filtrert = gdf_org[gdf_org[kolonne] == verdi]

    if gdf_filtrert.empty:
        print("Ingen treff på filteret!")
        return

    print(f"Lagrer filtrert data til: {output_fgb}")
    gdf_filtrert.to_file(output_fgb, driver="FlatGeobuf")

    # Les den nye filen (hele, ikke partial)
    gdf_ny, tid_ny = les_fil_med_tid(output_fgb)

    # Sammenligning
    print("\n Sammenligning:")
    print(f"- Original filstørrelse: {sjekk_filstørrelse(input_fgb)}")
    print(f"- Filtrert filstørrelse: {sjekk_filstørrelse(output_fgb)}")
    print(f"- Tid å lese originalfil: {tid_org:.2f} sekunder (med partial read)")
    print(f"- Tid å lese filtrert fil: {tid_ny:.2f} sekunder")
    print(f"- Antall rader etter filtrering: {len(gdf_filtrert)}")

# Parametere
input_fgb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb"
output_fgb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/filtered_arjordbr_24.fgb"
kolonne = "arjordbr"
verdi = 24

In [92]:

# Kjør funksjonen
filtrer_og_sammenlign(input_fgb, kolonne, verdi, output_fgb)


Leser originalfil (kun kolonne 'arjordbr') fra: /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb

Filtrerer på: arjordbr = 24
Lagrer filtrert data til: /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/filtered_arjordbr_24.fgb

 Sammenligning:
- Original filstørrelse: 275.65 MB
- Filtrert filstørrelse: 5.54 MB
- Tid å lese originalfil: 0.90 sekunder (med partial read)
- Tid å lese filtrert fil: 0.01 sekunder
- Antall rader etter filtrering: 3807


## Buffer og filtrering

In [71]:
import pyogrio
import geopandas as gpd
import folium
from shapely.geometry import Point
from IPython.display import display

def create_map_with_buffer_partialread_all(fgb_path, column, center_x, center_y, buffer_radius):
    """
    Lager et kart basert på AR50-data med partial read: Leser kun nødvendig kolonne + geometri.
    Viser alle objekter innenfor bufferområdet med fargekoding etter verdien i 'column'.
    Objekter som kun delvis ligger innenfor bufferen klippes slik at bare den delen som ligger
    innenfor bufferen vises. I tillegg lages separate lag for hver unik verdi slik at de kan
    slås av/på via LayerControl.

    Merk: Rader med verdien 98 i den angitte kolonnen blir ekskludert.

    Returnerer en tuple med:
      - folium Map, og
      - filtrert GeoDataFrame (tabell) med dataene.
    """
    # Les kun valgt kolonne + geometri
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])

    # Projiser til UTM (EPSG:25832) for korrekt bufferberegning
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffer
    center_point = Point(center_x, center_y)
    # Sentrer input (i EPSG:4326) og projiser til UTM
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]
    buffer_geom = center_point.buffer(buffer_radius)
    # Buffergeometri omprojiseres tilbake til EPSG:4326 for visning på kart
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Klipp ut (clip) objekter slik at vi får med den delen som ligger innenfor bufferen
    gdf_clipped = gpd.clip(gdf, buffer_geom)

    # Filtrer ut rader der verdien i 'column' er 98 (ikke registrerte områder)
    gdf_clipped = gdf_clipped[gdf_clipped[column] != 98]

    # For å vise data i en tabell, konverter GeoDataFrame til EPSG:4326
    table_data = gdf_clipped.to_crs(epsg=4326)

    # Bygg et fargekart basert på unike verdier i den angitte kolonnen
    unique_values = table_data[column].unique().tolist()
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'darkred', 'lightred', 'beige',
              'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink',
              'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    color_map = {val: colors[i % len(colors)] for i, val in enumerate(unique_values)}
    print("Fargekart:", color_map)

    # Lag kart med senterpunkt (i EPSG:4326)
    center = [center_y, center_x]
    m = folium.Map(location=center, zoom_start=12, tiles="cartodb positron")

    # Legg til bufferområdet som et lag
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "black", "weight": 1, "fillOpacity": 0.0},
    ).add_to(m)

    # Opprett et lag for hvert unikt verdinivå i 'column'
    for val in unique_values:
        subset = table_data[table_data[column] == val]
        # Lambdaen fanger den nåværende 'val' for korrekt fargelegging
        layer = folium.GeoJson(
            subset,
            name=f"{column}: {val}",
            style_function=lambda feature, val=val: {
                "color": color_map.get(val, "black"),
                "weight": 1,
                "fillOpacity": 0.4
            },
            tooltip=folium.GeoJsonTooltip(fields=[column], aliases=[f"{column}:"])
        )
        layer.add_to(m)

    folium.LayerControl().add_to(m)

    # Returner både kartet og dataene (tabellen)
    return m, table_data

## "Jeg ønsker å vite om top 10 bebygg (arealtype 10) i Kristiansand.(område = 4204) "

In [75]:

import geopandas as gpd
import folium
from IPython.display import display

def filter_ar50_with_Komid_map(ar50_path, filter_column, filter_value, komid_value=None, n_rows=10):
    """
    Leser AR50-data og filtrerer ut de radene der:
      - Verdien i filter_column (for eksempel "artype") er lik filter_value, og
      - Om komid_value ikke er None (eller 0), filtreres det også på at kolonnen "komid"
        er lik komid_value.

    Returnerer de første n_rows radene (som en GeoDataFrame) med kun de kolonnene:
      - filter_column,
      - "rid",
      - "komid" og
      - "geometry".
    """
    # Les AR50-data
    gdf = gpd.read_file(ar50_path)

    # Filtrer på valgt kolonne og verdi (f.eks. "artype" == 10)
    gdf_filtered = gdf[gdf[filter_column] == filter_value]

    # Dersom komid_value ikke er None (eller 0), filtrer videre på "komid"
    if komid_value is not None and komid_value != 0:
        gdf_filtered = gdf_filtered[gdf_filtered["komid"] == komid_value]

    # Definer ønskede kolonner
    desired_cols = [filter_column, "rid", "komid", "geometry"]
    # Sørg for at kun de kolonnene som eksisterer beholdes
    desired_cols = [col for col in desired_cols if col in gdf_filtered.columns]

    # Velg de ønskede kolonnene og returner de første n_rows radene
    return gdf_filtered[desired_cols].head(n_rows)



# Brukergensesnittet

# Hvilken fil ønsker du å filtrere etter?

In [59]:
ar50_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb"

## Her kan du legge inn din ønskede kolonne og verdi du vil filterer etter! Du kan også filterer etter hvilken postkode du vil filtrere etter og hvor mange rader du øsnker skal vises.

In [76]:
import folium
import geopandas as gpd

# Bruk den eksisterende funksjonen for å filtrere data
result_table_komid = filter_ar50_with_Komid_map(
    ar50_file,
    filter_column="artype",
    filter_value=10,
    komid_value=4204,
    n_rows=10
)
display(result_table_komid)
# Sørg for at dataene er i EPSG:4326 slik at de vises riktig i folium
if result_table_komid.crs and result_table_komid.crs != "EPSG:4326":
    result_table_komid = result_table_komid.to_crs("EPSG:4326")

# Finn et sentralt punkt for å plassere kartet.
# Vi regner ut den geometriske unionens tyngdepunkt for å definere sentrum.
center = result_table_komid.union_all().centroid
map_center = [center.y, center.x]

# Lag et folium-kart med et initialt zoomnivå
m = folium.Map(location=map_center, zoom_start=12)

# Legg til GeoDataFrame-en som en GeoJson-lag
folium.GeoJson(result_table_komid).add_to(m)

# Legg til et lagkontroll for å aktivere visning av lag hvis du skulle utvide kartet
folium.LayerControl().add_to(m)

# Vis kartet direkte i notebooken
m


,artype,rid,komid,geometry
50,10,54288,4204,"MULTIPOLYGON (((450134.628 6441551.831, 450134..."
60,10,60364,4204,"MULTIPOLYGON (((449029.688 6441498.263, 449030..."
80,10,54371,4204,"MULTIPOLYGON (((449711.474 6442082.689, 449698..."
108,10,60180,4204,"MULTIPOLYGON (((448979.183 6441979.725, 448977..."
131,10,53663,4204,"MULTIPOLYGON (((448932.586 6441800.214, 448935..."
210,10,58755,4204,"MULTIPOLYGON (((440547.038 6438311.187, 440561..."
271,10,51054,4204,"MULTIPOLYGON (((439712.295 6437953.629, 439715..."
277,10,51006,4204,"MULTIPOLYGON (((439746.715 6437315.475, 439777..."
285,10,50703,4204,"MULTIPOLYGON (((439653.736 6436813.732, 439654..."
371,10,56829,4204,"MULTIPOLYGON (((436772.962 6439314.314, 436756..."


## Her kan du sette et koordinat og velge hvor stor omrekts rundt du ønsker å filterer etter!

In [72]:

map_obj, filtered_table_buffer = create_map_with_buffer_partialread_all(
    fgb_path=ar50_file,
    column="artreslag",
    center_x=8.01122450699552,
    center_y=58.1743128719827,
    buffer_radius=1000
)

# Vis først tabellen med de filtrerte dataene
display(filtered_table_buffer)

# Deretter vises det interaktive kartet
display(map_obj)

Fargekart: {31: 'red', 33: 'blue', 32: 'green', 39: 'orange'}


,artreslag,geometry
1952,31,"POLYGON ((7.99709 58.16968, 7.997 58.16947, 7...."
1318,31,"POLYGON ((8.00861 58.16597, 8.0087 58.16593, 8..."
1936,31,"POLYGON ((8.00018 58.1676, 8.00007 58.16777, 7..."
1938,31,"POLYGON ((8.00124 58.16937, 8.00185 58.16976, ..."
1951,33,"POLYGON ((7.99906 58.16963, 7.99925 58.16951, ..."
...,...,...
2015,31,"POLYGON ((8.00628 58.18274, 8.00656 58.18264, ..."
2001,31,"MULTIPOLYGON (((8.00942 58.18302, 8.00938 58.1..."
2014,31,"POLYGON ((8.0072 58.18286, 8.0074 58.18269, 8...."
2003,32,"POLYGON ((8.01015 58.18324, 8.01003 58.18319, ..."
